# Clase 11: Procesamiento de datos espaciales 


- Geopandas
    - https://pygis.io/docs/a_intro.html
    - Datasets https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG

## Vectorial
- seguir: https://geopandas.org/en/stable/getting_started/introduction.html
- https://levelup.gitconnected.com/beginners-tutorial-on-how-to-use-python-s-geopandas-map-library-e6fb0db8132
- https://geopandas.org/en/stable/getting_started.html
- https://towardsdatascience.com/geopandas-hands-on-introduction-to-geospatial-machine-learning-6e7e4a539daf
- contextily: context geo tiles in Python: https://contextily.readthedocs.io/en/latest/index.html
-https://fossies.org/linux/plotly.py/doc/python/lines-on-maps.md

In [ ]:
import numpy as np
import shapely
import geopandas as gpd
import contextily as cx
import json

In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
gdf = gpd.read_file("datos/geo/linea_de_limite_070111.zip")

In [ ]:
gdf

In [ ]:
gdf.info()

In [ ]:
gdf.crs

In [ ]:
ax = gdf.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')


Before adding web map tiles to this plot, we first need to ensure the coordinate reference systems (CRS) of the tiles and the data match. **Web map tiles are typically provided in Web Mercator (EPSG 3857)**

In [ ]:
df_wm = gdf.to_crs(epsg=3857)


In [ ]:
ax = df_wm.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
# tenemos que armarnos los datos para plotly
lats = []
lons = []
names = []

for feature, name in zip(gdf.geometry, gdf.nam):
    linestrings = [feature]
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y)) # explicar esto
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

En caso que el dataframe tenga diferentes tipos de datos geo en la columna geometry debo chequear que sea linestring
```python
for feature, name in zip(gdf.geometry, gdf.nam):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue

    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)
```

In [ ]:
fig = go.Figure()

plot_rios = go.Scattermapbox(lon=lons, lat=lats, mode="lines")

fig.add_trace(plot_rios)

fig.update_layout(mapbox_style="open-street-map", 
                  mapbox=dict(center=dict(lat=-31.663, lon=-60.725),zoom=3),)

Veamos el caso de los ríos ahora, pero como son demasiados **lo truncamos!!**

In [ ]:
rios = gpd.read_file("datos/geo/lineas_de_aguas_continentales_perenne.zip")
rios

In [ ]:
# analizando lo que hay en cada fila

#ls = rios.geometry[0:1]
#x,y = ls[0].xy
#x = np.array(x)
#y = np.array(y)

In [ ]:
# me quedo con algunos para que no demore
rios_recorte = rios[:1000]

In [ ]:
# tenemos que armarnos los datos para plotly
lats = []
lons = []
names = []

for feature, name in zip(rios_recorte.geometry, rios_recorte.nam):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y)) # explicar esto
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)


In [ ]:
fig = go.Figure()

plot_rios = go.Scattermapbox(lon=lons, lat=lats, mode="lines")

fig.add_trace(plot_rios)

fig.update_layout(mapbox_style="open-street-map",
                        margin={"r":0,"t":0,"l":0,"b":0},
                        mapbox=dict(center=dict(lat=-31.663, lon=-60.725),zoom=9),
                        )

In [ ]:
muni = gpd.read_file("datos/geo/municipio.zip")

In [ ]:
muni

In [ ]:
muni.info()

In [ ]:
muni["aleatorio"] = np.random.randint(1,100,len(muni))
muni = muni[:150]

In [ ]:

#muni = muni.to_crs("WGS84") # con plotly no hace falta


fig = go.Figure()

plot_muni = go.Choroplethmapbox(geojson=json.loads(muni.to_json()), 
                                    locations=muni.index, z=muni['aleatorio'],
                                    colorscale="Viridis", marker_line_width=.5)

plot_rios = go.Scattermapbox(lon=lons, lat=lats, mode="lines")

fig.add_trace(plot_muni)
fig.add_trace(plot_rios)

fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=dict(lat=-31.663, lon=-60.725),zoom=5),)

**Ahora con plotly express**

```python
fig = px.choropleth_mapbox(muni,geojson=muni.geometry, locations=muni.index,
    color="aleatorio",
    center=dict(lat=-31.663, lon=-60.725),mapbox_style="open-street-map",
    zoom=9)

fig.update_layout(
    height=1000,
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig
```

In [ ]:
muni.area

In [ ]:
muni.crs

In [ ]:
muni = muni.to_crs(epsg=3857)

In [ ]:
muni.crs

In [ ]:
muni["geometry"].area

In [ ]:
muni["area"] = muni["geometry"].area/(10**6) # lo paso a km2

In [ ]:
muni

In [ ]:
muni = muni.to_crs(epsg=32633)
muni["area2"] = muni.geometry.area/(10**6)

In [ ]:
muni

In [ ]:
muni.to_crs({'proj':'cea'}) # cilindrical equal-area
muni.crs

In [ ]:
muni["area3"] = muni.area/(10**6)
muni

In [ ]:
muni = muni.to_crs("WGS84")
fig = go.Figure()

plot_muni = go.Choroplethmapbox(geojson=json.loads(muni.to_json()), 
                                    locations=muni.index, z=muni['area3'],
                                     marker_line_width=.5)



fig.add_trace(plot_muni)


fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=dict(lat=-31.663, lon=-60.725),zoom=5),)

In [ ]:
muni.centroid

In [ ]:
muni.explore("area3", legend=False)